In [287]:
!pip install emoji

### Working with emoji package

In [288]:
import emoji as emoji

In [289]:
# emoji.EMOJI_UNICODE --> to see all thr emojis

In [290]:
emoji_dictionary = {"0": "\u2764\uFE0F",    
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                    "5": ":hundred_points:",
                    "6": ":fire:",
                    "7": ":chestnut:"
                   }

In [291]:
emoji.emojize(":chestnut:")

'🌰'

In [292]:
for e in emoji_dictionary.values():           # print all emoji in the list
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴
💯
🔥
🌰


### Step2: Processing custom emoji dataset

In [293]:
import numpy as np
import pandas as pd

In [294]:
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [295]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [296]:
# Let us print the sentences with emojis 
data = train.values
print(data.shape)

(132, 4)


In [297]:
XT = train[0]
Xt = test[0]

YT = train[1]
Yt = test[1]

In [298]:
for i in range(5):
    print(XT[i],emoji.emojize(emoji_dictionary[str(YT[i])]))

never talk to me again 😓
I am proud of your achievements 😁
It is the worst day in my life 😓
Miss you so much ❤️
food is life 🍴


### Step 3: Converting sentences into Embeddings

In [299]:
embeddings = {}
with open('glove.6B.50d.txt',encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:],dtype='float32')
        
        embeddings[word] = coeffs

In [300]:
embeddings["eat"]

array([ 6.4295e-01, -4.2946e-01, -5.4277e-01, -1.0307e+00,  1.2056e+00,
       -2.7174e-01, -6.3561e-01, -1.5065e-02,  3.7856e-01,  4.6474e-02,
       -1.3102e-01,  6.0500e-01,  1.6391e+00,  2.3940e-01,  1.2128e+00,
        8.3178e-01,  7.3893e-01,  1.5200e-01, -1.4175e-01, -8.8384e-01,
        2.0829e-02, -3.2545e-01,  1.8035e+00,  1.0045e+00,  5.8484e-01,
       -6.2031e-01, -4.3296e-01,  2.3562e-01,  1.3027e+00, -8.1264e-01,
        2.3158e+00,  1.1030e+00, -6.0608e-01,  1.0101e+00, -2.2426e-01,
        1.8908e-02, -1.0931e-01,  3.8350e-01,  7.7362e-01, -8.1927e-02,
       -3.4040e-01, -1.5143e-03, -5.6640e-02,  8.7359e-01,  1.4805e+00,
        6.9421e-01, -3.0966e-01, -9.0826e-01,  3.7277e-03,  8.4550e-01],
      dtype=float32)

In [301]:
emb_dim = embeddings["eat"].shape[0]
print(emb_dim)

50


### Step  4: Converting Sentences into Vector (Embedding layer output)

In [302]:
def getOutputEmbeddings(X):
    
    embedding_matrix_output = np.zeros((X.shape[0],10,50))
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for jx in range(len(X[ix])):
            embedding_matrix_output[ix][jx] = embeddings[X[ix][jx].lower()]
            
    return embedding_matrix_output

In [303]:
emb_XT = getOutputEmbeddings(XT)
emb_Xt = getOutputEmbeddings(Xt)

<ipython-input-302-73145de5bfac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [304]:
print(emb_XT.shape)
print(emb_Xt.shape)

(132, 10, 50)
(56, 10, 50)


In [305]:
from keras.utils import to_categorical

In [306]:
YT=to_categorical(YT,num_classes=5)
Yt=to_categorical(Yt,num_classes=5)

### Step 5: Define the RNN/LTSM Model

In [307]:
from keras.layers import *
from keras.models import Sequential

In [308]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50),return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_37 (LSTM)               (None, 10, 64)            29440     
_________________________________________________________________
dropout_37 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_38 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_38 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 5)                 325       
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
_________________________________________________

In [309]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

checkpoint= ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)
earlystop= EarlyStopping(monitor='val_acc',patience=10)

hist = model.fit(emb_XT,YT,batch_size=32,epochs=100 ,shuffle=True,validation_split=0.1)

Epoch 1/100
4/4 [==============================] - 2s 495ms/step - loss: 1.5983 - accuracy: 0.1864 - val_loss: 1.6037 - val_accuracy: 0.2857
Epoch 2/100
4/4 [==============================] - 0s 19ms/step - loss: 1.5534 - accuracy: 0.3220 - val_loss: 1.6426 - val_accuracy: 0.2857
Epoch 3/100
4/4 [==============================] - 0s 17ms/step - loss: 1.4899 - accuracy: 0.3305 - val_loss: 1.6727 - val_accuracy: 0.3571
Epoch 4/100
4/4 [==============================] - 0s 28ms/step - loss: 1.5292 - accuracy: 0.3729 - val_loss: 1.6738 - val_accuracy: 0.2143
Epoch 5/100
4/4 [==============================] - 0s 27ms/step - loss: 1.4746 - accuracy: 0.4153 - val_loss: 1.6453 - val_accuracy: 0.1429
Epoch 6/100
4/4 [==============================] - 0s 28ms/step - loss: 1.4357 - accuracy: 0.4237 - val_loss: 1.5971 - val_accuracy: 0.2143
Epoch 7/100
4/4 [==============================] - 0s 40ms/step - loss: 1.3872 - accuracy: 0.4322 - val_loss: 1.5155 - val_accuracy: 0.2143
Epoch 8/100
4/4 [==

In [310]:
pred = model.predict_classes(emb_Xt)

In [311]:
print(pred)

[4 3 3 2 2 2 3 2 4 2 1 2 0 3 1 3 2 2 3 2 2 0 4 3 3 3 2 0 1 2 0 1 3 2 0 1 2
 3 4 2 1 0 0 1 2 2 3 2 0 3 3 0 3 2 2 3]


In [312]:
model.evaluate(emb_Xt,Yt)

2/2 [==============================] - 0s 10ms/step - loss: 2.1056 - accuracy: 0.6250


[2.1055965423583984, 0.625]

In [313]:
pred = model.predict_classes(emb_Xt)

In [314]:
for i in range(30):
    print(' '.join(Xt[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Yt[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😓
😓
he got a raise
😁
😓
she got me a present
❤️
😁
ha ha ha it was so funny
😁
😁
he is a good friend
❤️
😁
I am upset
❤️
😓
We had such a lovely dinner tonight
❤️
😁
where is the food
🍴
🍴
Stop making this joke ha ha ha
😁
😁
where is the ball
⚾
⚾
work is hard
😓
😁
This girl is messing with me
😓
❤️
are you serious ha ha
😁
😓
Let us go play baseball
⚾
⚾
This stupid grader is not working
😓
😓
work is horrible
😓
😁
Congratulation for having a baby
😁
😁
stop messing around
😓
😓
any suggestions for dinner
🍴
😁
I love taking breaks
❤️
😁
you brighten my day
😁
❤️
I boiled rice
🍴
🍴
she is a bully
😓
😓
Why are you feeling bad
😓
😓
I am upset
😓
😓
I worked during my birthday
😓
😁
My grandmother is the love of my life
❤️
❤️
enjoy your break
😁
⚾
valentine day is near
❤️
😁
